# Metoda Gaussa-Jordana

In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import time

def print_system(A, B):
    for i in range(len(B)):
        print(f"{A[i]}\t[{B[i]}]")

def gauss_jordan(A, B):
    n = len(B)
    npA = np.array(A)
    npB = np.array(B)
    
    max_value = np.max(A)
    for i in range(n):
        for j in range(n):
            npA[i][j] /= max_value
        npB[i] /= max_value
    
    for start_ind in range(n):
        max_row = max(range(start_ind, n), key=lambda r: abs(npA[r][start_ind]))
        npA[[max_row, start_ind]] = npA[[start_ind, max_row]]
        npB[[max_row, start_ind]] = npB[[start_ind, max_row]]
        for col in range(start_ind + 1, n):
            npA[start_ind][col] /= npA[start_ind][start_ind]
        npB[start_ind] /= npA[start_ind][start_ind]
        npA[start_ind][start_ind] = 1.0
        for row in range(n):
            if row != start_ind:
                for col in range(start_ind + 1, n):
                    npA[row][col] -= npA[start_ind][col] * npA[row][start_ind]
                npB[row] -= npB[start_ind] * npA[row][start_ind]
                npA[row][start_ind] = 0.0
    return (npA, npB)

In [4]:
A = np.random.uniform(10, 500, size=(600,600))
B = np.random.uniform(10, 500, size=(600,1))

start_time = time.time()
(gjA, gjB) = gauss_jordan(A,B)
end_time = time.time()

print(f"Gauss-Jordan: {end_time - start_time}")

start_time = time.time()
np_solved = np.linalg.solve(np.array(A), np.array(B))
end_time = time.time()

print(f"Numpy: {end_time - start_time}")

print("________________________________")
print("Różnica wyników pomiędzy zaimplementowanym Gaussem-Jordanem, a numpy.linalg.solve")
print(np.max((np_solved - gjB) / np_solved))

Gauss-Jordan: 94.77914547920227
Numpy: 0.048226356506347656
________________________________
Różnica wyników pomiędzy zaimplementowanym Gaussem-Jordanem, a numpy.linalg.solve
2.1498894649746205e-10


In [3]:
def LU(A_matrix):
    A = A_matrix.copy()
    n = len(A)
    
    # Pivoting matrix
    
    L = np.identity(n)
    U = np.zeros((n,n))
    
    P = np.identity(n)
    
    for j in range(n):
        max_row = max(range(j, n), key=lambda i: abs(A[i][j]))
        P[[max_row, j]] = P[[j, max_row]]
        
    PA = P @ A
    
    for j in range(n):
        for i in range(j+1):
            U[i][j] = PA[i][j] - sum(U[k][j] * L[i][k] for k in range(i))

        for i in range(j+1, n):
            L[i][j] = (PA[i][j] - sum(U[k][j] * L[i][k] for k in range(j))) / U[j][j]

    return (P, L, U)

In [8]:
A = np.random.uniform(100, 40000, size=(300,300))

start_time = time.time()
(P, Lower, Upper) = LU(A)
end_time = time.time()

print("My LU:")
print(f"Time: {end_time - start_time}s")     
print(f"L:\n{Lower}")
print(f"U:\n{Upper}")

print()
print("max(PA - LU):")
print(np.amax(np.abs(P @ A - Lower @ Upper)))

start_time = time.time()
(Pi, Lo, Up) = sp.linalg.lu(A)
end_time = time.time()

print("SciPy LU:")
print(f"Time: {end_time - start_time}s")

My LU:
Time: 5.8890955448150635s
L:
[[ 1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.91430613  1.          0.         ...  0.          0.
   0.        ]
 [ 0.87656814  0.68522743  1.         ...  0.          0.
   0.        ]
 ...
 [ 0.9138632  -0.26975329  1.8975891  ...  1.          0.
   0.        ]
 [ 0.16333468  3.87136563 -7.11308605 ... -2.74821968  1.
   0.        ]
 [ 0.68599804 -0.60623002  3.07383237 ... -3.25782282  1.99326362
   1.        ]]
U:
[[  39934.04087294   36171.4919645     2454.71552004 ...   14491.12117875
    19368.55347805    5702.05056523]
 [      0.            6744.6864895    31694.19367517 ...   -3058.15741674
    -5177.19079527   10804.67997024]
 [      0.               0.           16009.98111353 ...   16409.46889529
     3757.60595306   -5523.29636038]
 ...
 [      0.               0.               0.         ...  -43629.67282076
   -54977.58200206  -85033.99431376]
 [      0.               0.               0.         ...    